In [1]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle

In [2]:
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time

# Scrape Data and Organize

In [4]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [5]:
# drop any items in training data collection
db.training_data.drop()

In [6]:
# Url to get all boxscores for 2017-2018 regular season games
nba_17_18_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/?Season=2017-18&SeasonType=Regular%20Season"
nba_17_18_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/?Season=2017-18&SeasonType=Regular%20Season"

In [7]:
# create splinter browser
def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    return browser

browser = init_browser()

In [8]:
# visit website
browser.visit(nba_17_18_boxscores_url)
time.sleep(3)

In [9]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])
# Rename column names
new_df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
new_df['GameDate'] = new_df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
# export to json
#items = new_df.to_json(orient='records', date_format='iso')
# load json string to json
#items_db = json.loads(items)

In [10]:
# visit website
browser.visit(nba_17_18_fourfactor_url)
time.sleep(3)

In [11]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
# export to json
#items = new_df.to_json(orient='records', date_format='iso')
# load json string to json
#items_db = json.loads(items)

In [12]:
df = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [13]:
len(df)

2460

In [14]:
result = pd.merge(new_df, df, on=['Team', 'GameDate'])

In [15]:
result.columns

Index(['Team', 'MatchUp', 'GameDate', 'W/L', 'MIN', 'OffRtg', 'DefRtg',
       'NetRtg', 'AST%', 'AST/TO', 'ASTRatio', 'OREB%', 'DREB%', 'REB%',
       'TOV%', 'eFG%', 'TS%', 'PACE', 'PIE', 'FTARate', 'OppeFG%',
       'OppFTARate', 'OppTOV%', 'OppOREB%'],
      dtype='object')

In [16]:
result_training_df = result.copy()

In [17]:
nba_18_19_boxscores_url = "https://stats.nba.com/teams/boxscores-advanced/"
nba_18_19_fourfactor_url = "https://stats.nba.com/teams/boxscores-four-factors/"

In [18]:
# drop any items in training data collection
db.testing_data.drop()

In [19]:
# visit website
browser.visit(nba_18_19_boxscores_url)
time.sleep(3)

In [20]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
new_df = df[0].drop(columns=['Season'])
# Rename column names
new_df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp"}, inplace=True)
# change gamedate to datetime
new_df['GameDate'] = new_df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
# export to json
#items = new_df.to_json(orient='records', date_format='iso')
# load json string to json
#items_db = json.loads(items)

In [21]:
# visit website
browser.visit(nba_18_19_fourfactor_url)
time.sleep(3)

In [22]:
# get click path by xpath from browser inspector
sel_all_path = browser.find_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]")
# select to display all games
sel_all_path.click()
time.sleep(5)
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# drop season column since unneeded
df = df[0].drop(columns=['Season'])
# Rename columns
df.rename(columns={"Game\xa0Date": "GameDate", "Match\xa0Up": "MatchUp", "OppFTA\xa0Rate": "OppFTARate"}, inplace=True)
# Convert to float percent values
df['OREB%'] = df['OREB%'].str.rstrip('%').astype('float')
df['OppOREB%'] = df['OppOREB%'].str.rstrip('%').astype('float')
df['OppeFG%'] = df['OppeFG%'].str.rstrip('%').astype('float')
df['eFG%'] = df['eFG%'].str.rstrip('%').astype('float')
# change gamedate to datetime
df['GameDate'] = df['GameDate'].apply(lambda x: dt.datetime.strptime(x, "%m/%d/%Y"))
# export to json
#items = new_df.to_json(orient='records', date_format='iso')
# load json string to json
#items_db = json.loads(items)

In [23]:
df = df.drop(['MIN', 'MatchUp', 'W/L', 'eFG%', 'OREB%', 'TOV%'], axis=1)

In [24]:
result_new = pd.merge(new_df, df, on=['Team', 'GameDate'])

In [25]:
len(result_new)

1438

In [26]:
df_winloss = pd.concat([result, result_new])

In [27]:
df_winloss = df_winloss.groupby(['W/L'])['AST%', 'AST/TO', 'ASTRatio', 'DREB%', 
                                     'DefRtg','OREB%', 'OffRtg', 'PACE', 
                                     'PIE', 'REB%', 'TOV%','TS%',
                                     'eFG%', 'FTARate', 'OppFTARate', 'OppOREB%',
                                     'OppTOV%', 'OppeFG%'].mean()

In [28]:
df_winloss = df_winloss.reset_index()

In [30]:
db.nba_stats_data.drop()

In [31]:
# export to json
items = df_winloss.to_json(orient='records')
# load json string to json
items_db = json.loads(items)

In [32]:
# insert data to collection
db.nba_stats_data.insert_many(items_db)

In [33]:
df_winloss.head()

,W/L,AST%,AST/TO,ASTRatio,DREB%,DefRtg,OREB%,OffRtg,PACE,PIE,REB%,TOV%,TS%,eFG%,FTARate,OppFTARate,OppOREB%,OppTOV%,OppeFG%
0,L,57.715854,1.618132,16.250795,72.448743,113.929400,25.942227,102.535403,99.069733,43.054643,48.119035,14.767111,52.923037,49.366290,0.247958,0.269453,27.552232,14.003592,55.202514
1,W,59.911339,1.959913,18.224885,74.059312,102.535403,27.552232,113.929400,99.069733,56.945818,51.881734,14.003592,58.682914,55.202514,0.269453,0.247958,25.942227,14.767111,49.366290


In [34]:
result['OffEff'] = np.where(result['OffRtg']>=df_winloss.loc[1,'OffRtg'], 1, 0)
result['DefEff'] = np.where(result['DefRtg']<=df_winloss.loc[1,'DefRtg'], 1, 0)

In [35]:
result_new['OffEff'] = np.where(result_new['OffRtg']>=df_winloss.loc[1,'OffRtg'], 1, 0)
result_new['DefEff'] = np.where(result_new['DefRtg']<=df_winloss.loc[1,'DefRtg'], 1, 0)

In [36]:
# export to json
items = result.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [37]:
# insert data to collection
db.training_data.insert_many(items_db)

In [38]:
# export to json
items = result_new.to_json(orient='records', date_format='iso')
# load json string to json
items_db = json.loads(items)

In [39]:
# insert data to collection
db.testing_data.insert_many(items_db)

# Create Four Factor Model

In [40]:
temp = db.training_data.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)

In [41]:
df = pd.DataFrame(temp)
df.head()

,AST%,AST/TO,ASTRatio,DREB%,DefEff,DefRtg,FTARate,GameDate,MIN,MatchUp,...,OppTOV%,OppeFG%,PACE,PIE,REB%,TOV%,TS%,Team,W/L,eFG%
0,35.5,0.79,9.6,77.3,0,104.3,0.146,2018-04-11T00:00:00.000Z,48,HOU @ SAC,...,12.0,51.9,92.0,30.3,46.4,15.2,47.5,HOU,L,45.7
1,50.0,1.69,16.1,78.0,0,131.7,0.264,2018-04-11T00:00:00.000Z,48,MEM @ OKC,...,13.5,67.1,104.0,44.2,48.3,12.5,60.6,MEM,L,55.5
2,67.3,2.92,21.7,77.4,1,88.8,0.143,2018-04-11T00:00:00.000Z,48,PHI vs. MIL,...,18.7,45.0,107.0,67.1,55.0,11.2,58.2,PHI,W,56.7
3,59.5,1.29,15.6,71.8,0,116.2,0.207,2018-04-11T00:00:00.000Z,48,SAS @ NOP,...,18.1,61.9,105.5,34.8,43.8,16.0,48.8,SAS,L,44.6
4,69.8,2.00,21.7,58.1,1,102.1,0.153,2018-04-11T00:00:00.000Z,48,NYK @ CLE,...,16.7,42.6,97.0,62.1,45.2,15.3,60.6,NYK,W,57.6


In [97]:
X = df.loc[:,['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y = df['W/L']

In [98]:
# Split the data using train_test_split
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [99]:
X_scaler = StandardScaler().fit(X)
X_train_scaled = X_scaler.transform(X)

In [100]:
from sklearn.externals import joblib

In [101]:
scaler_filename = "fourfactor_scaler.save"
joblib.dump(X_scaler, scaler_filename)

['fourfactor_scaler.save']

In [93]:
# Create a logistic regression model
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
fourfactor_model = LogisticRegression()
fourfactor_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
# Fit the model to the data
# YOUR CODE HERE
fourfactor_model.fit(X_train_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [47]:
fourfactor_model.coef_

array([[ 4.91329346,  0.99873369, -2.22702119,  2.01833405, -0.99873369,
        -2.01833405,  2.22702119, -4.91329346]])

In [48]:
temp = db.testing_data.find()
#temp = db.testing_data.find({'Team': 'LAL'})
temp = list(temp)
for i in temp:
    i.pop('_id', None)

In [49]:
df_test = pd.DataFrame(temp)
df_test.head()

,AST%,AST/TO,ASTRatio,DREB%,DefEff,DefRtg,FTARate,GameDate,MIN,MatchUp,...,OppTOV%,OppeFG%,PACE,PIE,REB%,TOV%,TS%,Team,W/L,eFG%
0,66.0,2.36,20.9,80.0,0,106.4,0.149,2019-01-24T00:00:00.000Z,48,OKC vs. NOP,...,11.0,53.5,109.5,52.8,55.0,12.7,56.7,OKC,W,56.4
1,65.9,2.90,19.9,79.5,0,121.2,0.221,2019-01-24T00:00:00.000Z,48,WAS vs. GSW,...,14.4,60.8,104.5,41.4,46.2,9.5,56.6,WAS,L,54.2
2,56.8,1.47,17.2,73.2,0,102.9,0.213,2019-01-24T00:00:00.000Z,48,POR @ PHX,...,15.5,48.3,103.0,60.1,55.7,16.5,58.4,POR,W,53.2
3,52.8,1.19,13.9,64.0,0,116.5,0.307,2019-01-24T00:00:00.000Z,48,PHX vs. POR,...,16.5,53.2,103.0,39.9,44.3,15.5,53.1,PHX,L,48.3
4,68.0,2.27,23.3,77.8,0,112.4,0.239,2019-01-24T00:00:00.000Z,48,GSW @ WAS,...,9.5,54.2,104.5,58.6,53.8,14.4,64.8,GSW,W,60.8


In [50]:
X = df_test.loc[:,['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y = df_test['W/L']

In [51]:
X_test_scaled = X_scaler.transform(X)

In [52]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# YOUR CODE HERE
predictions = fourfactor_model.predict(X_test_scaled)

In [53]:
len(predictions)

1438

In [54]:
# Print the r2 score for the test data
# YOUR CODE HERE
fourfactor_model.score(X_test_scaled, y)

0.9346314325452016

In [95]:
fourfactor_model.score(X_train_scaled, y)

0.9398373983739837

In [55]:
predictwincount = 0
actualwincount = 0
for x in range (len(predictions)):
    if(predictions[x] == 'W'):
        predictwincount += 1
    if(y[x] == 'W'):
        actualwincount += 1
print(f"Predicted W is {predictwincount}, actual is {actualwincount}")

Predicted W is 719, actual is 719


# Create next predictor

## Try using PACE

In [102]:
#X = result_new.loc[:,['TS%', 'FTARate', 'TOV%', 'OREB%', 'AST%', 'DREB%', 'PIE','OffEff', 'DefEff']]
#X = result_new.loc[:,['TS%', 'FTARate', 'TOV%', 'OREB%', 'AST%', 'DREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%',
#       'OppTOV%', 'OppeFG%']]
X = df.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%', 'PACE']]
y = df['W/L']

In [103]:
X_scaler = StandardScaler().fit(X)
X_train_scaled = X_scaler.transform(X)

In [58]:
newModel = LogisticRegression()

In [59]:
newModel.fit(X_train_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [60]:
X_test = df_test.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%', 'PACE']]
y_test = df_test['W/L']

In [64]:
X_test_scaled = X_scaler.transform(X_test)

In [90]:
# Print the r2 score for the test data
# YOUR CODE HERE
newModel.score(X_train_scaled, y)

0.9536585365853658

In [ ]:
newModel.score(X_test_scaled, y_test)

In [66]:
newModel.coef_

array([[ 5.708445  , -2.57058962,  2.40071017,  0.32193758,  0.85914937,
        -1.08230245, -2.11199394,  2.2960698 , -5.17168816,  0.08214409]])

In [67]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# YOUR CODE HERE
predictions = newModel.predict(X_test_scaled)

In [69]:
predictwincount = 0
actualwincount = 0
for x in range (len(predictions)):
    if(predictions[x] == 'W'):
        predictwincount += 1
    if(y_test[x] == 'W'):
        actualwincount += 1
print(f"Predicted W is {predictwincount}, actual is {actualwincount}")

Predicted W is 715, actual is 719


In [104]:
X = df.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y = df['W/L']

In [105]:
X_scaler = StandardScaler().fit(X)
X_train_scaled = X_scaler.transform(X)

In [106]:
myscaler_filename = "my_scaler.save"
joblib.dump(X_scaler, myscaler_filename)

['my_scaler.save']

In [81]:
newModel = LogisticRegression()

In [82]:
newModel.fit(X_train_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [83]:
X_test = df_test.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y_test = df_test['W/L']

In [84]:
X_test_scaled = X_scaler.transform(X_test)

In [85]:
newModel.coef_

array([[ 5.71346941, -2.56418493,  2.38773285,  0.31699128,  0.85398057,
        -1.06870842, -2.1274397 ,  2.30208167, -5.16829403]])

In [86]:
# Make predictions using the X_test and y_test data
# Print at least 10 predictions vs their actual labels
# YOUR CODE HERE
predictions = newModel.predict(X_test_scaled)

In [87]:
predictwincount = 0
actualwincount = 0
for x in range (len(predictions)):
    if(predictions[x] == 'W'):
        predictwincount += 1
    if(y_test[x] == 'W'):
        actualwincount += 1
print(f"Predicted W is {predictwincount}, actual is {actualwincount}")

Predicted W is 708, actual is 719


In [89]:
newModel.score(X_train_scaled, y)

0.9536585365853658

In [96]:
mymodel_filename = 'mymodel.pkl'
pickle.dump(newModel, open(mymodel_filename, 'wb'))

In [ ]:
fourfactor_filename = 'fourfactor.pkl'
pickle.dump(newModel, open(mymodel_filename, 'wb'))

# Create team name abreviation list

In [107]:
temp = db.testing_data.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df = pd.DataFrame(temp)

In [108]:
teams = df['Team'].unique()

In [109]:
myteam = [team for team in teams]

In [110]:
myteam.sort()

In [111]:
item = {'Teams': myteam}

In [112]:
item['Teams']

['ATL',
 'BKN',
 'BOS',
 'CHA',
 'CHI',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHX',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

In [113]:
db.nba_teams_data.drop()

In [114]:
db.nba_teams_data.insert_one(item)